<a href="https://colab.research.google.com/github/agntgalahad/anime-face-generator/blob/main/Anime_Face_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import cv2
from glob import glob
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import tensorflow as tf
!pip install kaggle 
from google.colab import files
from keras.layers import *
from keras.models import Model
from keras.optimizers import Adam

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
w_init = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"himan1234","key":"8061c18115e8805ead9e9a4249d477a1"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d soumikrakshit/anime-faces

100% 440M/441M [00:19<00:00, 25.3MB/s]
100% 441M/441M [00:20<00:00, 23.1MB/s]


In [ ]:
import zipfile

with zipfile.ZipFile('/content/anime-faces.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [ ]:
def load_images(image_path):
  image = tf.io.read_file(image_path)
  image = tf.io.decode_png(image)
  image = tf.image.resize(image, (img_h, img_w))
  image = tf.cast(image, tf.float32)
  image = (image-127.5)/127.5
  return image

In [ ]:
def tf_dataset(image_path, batch_size):
  dataset = tf.data.Dataset.from_tensor_slices(image_path)
  dataset = dataset.shuffle(buffer_size = 1024)
  dataset = dataset.map(load_images, 
                      num_parallel_calls = tf.data.experimental.AUTOTUNE)
  dataset = dataset.batch(batch_size)
  dataset = dataset.prefetch(buffer_size = tf.data.experimental.AUTOTUNE)
  return dataset

In [ ]:
def deconv_block(inputs, num_filters, kernel_size, strides, bn=True):
  x = Conv2DTranspose(filters=num_filters,
                      kernel_size=kernel_size, 
                      kernel_initializer=w_init,
                      padding='same',
                      strides = strides,
                      use_bias=False)(inputs)

  if bn:
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
  return x

In [ ]:
def conv_block(inputs, num_filters, kernel_size, padding="same", strides=2, activation=True):
    x = Conv2D(
        filters=num_filters,
        kernel_size=kernel_size,
        kernel_initializer=w_init,
        padding=padding,
        strides=strides,
    )(inputs)

    if activation:
        x = LeakyReLU(alpha=0.2)(x)
        x = Dropout(0.3)(x)
    return x

In [ ]:
def build_generator(latent_dim):
    f = [2**i for i in range(5)][::-1]
    filters = 32
    output_strides = 16
    h_output = img_h // output_strides
    w_output = img_w // output_strides

    noise = Input(shape=(latent_dim,), name="generator_noise_input")

    x = Dense(f[0] * filters * h_output * w_output, use_bias=False)(noise)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Reshape((h_output, w_output, 16 * filters))(x)

    for i in range(1, 5):
        x = deconv_block(x,
            num_filters=f[i] * filters,
            kernel_size=5,
            strides=2,
            bn=True
        )

    x = conv_block(x,
        num_filters=3,  ## Change this to 1 for grayscale.
        kernel_size=5,
        strides=1,
        activation=False
    )
    fake_output = Activation("tanh")(x)

    return Model(noise, fake_output, name="generator")

In [ ]:
f = [2**i for i in range(5)][::-1]
print(f)

[16, 8, 4, 2, 1]


In [ ]:
def build_discriminator():
  f = [2**i for i in range(4)]
  filters = 64
  output_strides = 16
  h_output = img_h//output_strides
  w_output = img_w//output_strides

  image_input = Input(shape=(img_h, img_w, img_c))
  x = image_input
  for i in range(0,4):
    x = conv_block(x,
                   num_filters=f[i]*filters,
                   kernel_size=5,
                   strides=2)
    x = Flatten()(x)
    x = Dense(1)(x)

    return Model(image_input, x, name = 'discriminator')

In [ ]:
class GAN(Model):
  def __init__(self, discriminator, generator, latent_dim):
    super(GAN, self).__init__()
    self.discriminator = discriminator
    self.generator = generator
    self.latent_dim = latent_dim

  def compile(self, d_optimizer, g_optimizer, loss_fn):
    super(GAN, self).compile()
    self.d_optimizer = d_optimizer
    self.g_optimizer = g_optimizer
    self.loss_fn = loss_fn

  def train_step(self, real_images):
    batch_size = tf.shape(real_images)[0]

    for i in range(2):

      #training discriminator on fake images
      random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
      generated_images = self.generator(random_latent_vectors)
      generated_labels = tf.zeros((batch_size, 1))

      with tf.GradientTape() as ftape:
        predictions = self.discriminator(generated_images)
        dl_loss = self.loss_fn(generated_labels, predictions)
      grads = ftape.gradient(dl_loss, self.discriminator.trainable_weights)
      self.d_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_weights))
      #labels for real images
      labels = tf.ones((batch_size, 1))

      with tf.GradientTape() as rtape:
        predictions = self.discriminator(real_images)
        d2_loss = self.loss_fn(labels, predictions)
      grads = rtape.gradient(d2_loss, self.discriminator.trainable_weights)
      self.d_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_weights))
      
      random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
      misleading_labels = tf.ones((batch_size, 1))

      with tf.GradientTape() as gtape:
        predictions = self.discriminator(self.generator(random_latent_vectors))
        g_loss = self.loss_fn(misleading_labels, predictions)
      grads = gtape.gradient(g_loss, self.generator.trainable_weights)
      self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

      return {"dl_loss": dl_loss,  "d2_loss": d2_loss, "g_loss": g_loss}



In [ ]:
!mkdir samples
def save_plot(examples, epoch, n):
  examples = (examples + 1)/2.0
  for i in range(n*n):
    plt.subplot(n, n, i+1)
    plt.axis("off")
    plt.imshow(examples[i])
  
  file_name = f"samples/fake_image_epoch_{epoch+1}.png"
  plt.savefig(file_name)
  plt.close()

In [ ]:
!mkdir saved_model

In [ ]:
img_h = 64
img_w = 64
img_c = 3

if __name__ == "__main__":
  batch_size = 128
  latent_dim = 128
  epochs = 1000
  images_path = glob("/content/data/data/*")
  dataset = tf_dataset(images_path, batch_size)
  d_model = build_discriminator()
  g_model = build_generator(latent_dim)
  d_model.summary()
  g_model.summary()

  gan = GAN(d_model, g_model, latent_dim)
  bce_loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True, label_smoothing=0.1)
  d_optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
  g_optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
  gan.compile(d_optimizer, g_optimizer, bce_loss_fn)
  image_dataset = tf_dataset(images_path, batch_size)

  for epoch in range(epochs):
    gan.fit(image_dataset, epochs=1)
    g_model.save("saved_model/g_model.h5")
    d_model.save("saved_model/d_model.h5")

    n_samples = 25
    noise = np.random.normal(size=(n_samples, latent_dim))
    examples = g_model.predict(noise)
    save_plot(examples, epoch, int(np.sqrt(n_samples)))

Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 64)        4864      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 32, 32, 64)        0         
                                                                 
 dropout (Dropout)           (None, 32, 32, 64)        0         
                                                                 
 flatten (Flatten)           (None, 65536)             0         
                                                                 
 dense (Dense)               (None, 1)                 65537     
                                                                 
Total params: 70,401
Trainable params: 70,401
Non-tra

169/169 [==============================] - 23s 135ms/step - dl_loss: 0.2037 - d2_loss: 0.2172 - g_loss: 2.8368


169/169 [==============================] - 23s 138ms/step - dl_loss: 0.2025 - d2_loss: 0.2114 - g_loss: 2.8304


169/169 [==============================] - 24s 141ms/step - dl_loss: 0.2015 - d2_loss: 0.2085 - g_loss: 2.8353


169/169 [==============================] - 24s 142ms/step - dl_loss: 0.2012 - d2_loss: 0.2066 - g_loss: 2.8400


...

169/169 [==============================] - 24s 141ms/step - dl_loss: 0.2027 - d2_loss: 0.2001 - g_loss: 2.9217


169/169 [==============================] - 24s 139ms/step - dl_loss: 0.2026 - d2_loss: 0.2002 - g_loss: 2.9098


169/169 [==============================] - 24s 142ms/step - dl_loss: 0.2022 - d2_loss: 0.2001 - g_loss: 2.9116


169/169 [==============================] - 24s 141ms/step - dl_loss: 0.2020 - d2_loss: 0.2002 - g_loss: 2.9007


136/169 [=======================>......] - ETA: 4s - dl_loss: 0.2013 - d2_loss: 0.2000 - g_loss: 2.8917